# Entrenamiento clasificador unfrozen


In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)

In [ ]:
# Cargar datos
TRAIN_PATH = "datasets/EDDD/train"
VALID_PATH = "datasets/EDDD/val"
TEST_PATH = "datasets/EDDD/test"
BATCH_SIZE = 32
IMG_SIZE = (260, 260)
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    VALID_PATH,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_PATH,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Clases:", class_names)

In [ ]:
# Preprocesamiento
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Definir modelo
MODULE_HANDLE = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b2/feature_vector/2"

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMG_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=True), # 'trainable=True' para que esté unfrozen
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.build((None, *IMG_SIZE, 3))
model.summary()


In [ ]:
# Compilar y entrenar
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


In [ ]:
# Evaluar en test
loss, accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {accuracy*100:.2f}%")

In [ ]:
# Guardar modelo
SAVE_MODEL_DIR = "exported-unfrozen"
os.makedirs(SAVE_MODEL_DIR, exist_ok=True)
model.save(os.path.join(SAVE_MODEL_DIR, "efficientnet_v2_b2_unfrozen.h5"))
